In [ ]:
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U peft==0.8.2
!pip3 install -q -U trl==0.7.10
!pip3 install -q -U accelerate==0.27.1
!pip3 install -q -U datasets==2.17.0
!pip3 install -q -U transformers==4.38.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.4/183.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 32.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.7/279.7 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 47.5 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM

In [ ]:
from transformers import (
    HfArgumentParser,
    TrainingArguments,
    logging,
)

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
model_name = "google/gemma-2b-it"

################################################################################
# LoRA parameters
################################################################################
# LoRA attention dimension
lora_r = 64
# Alpha parameter for LoRA scaling
lora_alpha = 32
# Dropout probability for LoRA layers
lora_dropout = 0.05

################################################################################
# bitsandbytes parameters
################################################################################
# Activate 4-bit precision base model loading
use_4bit = True
# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"
# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"
# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

################################################################################
# TrainingArguments parameters
################################################################################
# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"
# Number of training epochs
num_train_epochs = 1
# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False
# Batch size per GPU for training
per_device_train_batch_size = 4
# Batch size per GPU for evaluation
per_device_eval_batch_size = 4
# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1
# Enable gradient checkpointing
gradient_checkpointing = True
# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3
# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4
# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001
# Optimizer to use
optim = "paged_adamw_32bit"
# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"
# Number of training steps (overrides num_train_epochs)
max_steps = -1
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
# Save checkpoint every X updates steps
save_steps = 25
# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################
# Pack multiple short examples in the same input sequence to increase efficiency
packing = True # False
# Load the entire model on the GPU 0
# device_map = {"": 0}
device_map="auto"


In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit, # Activates 4-bit precision loading
    bnb_4bit_quant_type=bnb_4bit_quant_type, # nf4
    bnb_4bit_compute_dtype=compute_dtype, # float16
    bnb_4bit_use_double_quant=use_nested_quant, # False
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name,
                                          trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.16k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/888 [00:00<?, ?B/s]

In [ ]:
gujarati_text = """ગૂગલ ન્યૂઝ એ ગૂગલ દ્વારા વિકસિત ન્યૂઝ એગ્રીગેટર સેવા છે. તે હજારો પ્રકાશકો અને સામયિકોમાંથી આયોજિત લેખોની લિંક્સનો સતત પ્રવાહ રજૂ કરે છે. Google News એ Android, iOS અને વેબ પર એપ્લિકેશન તરીકે ઉપલબ્ધ છે. ગૂગલે સપ્ટેમ્બર 2002માં બીટા વર્ઝન અને જાન્યુઆરી 2006માં સત્તાવાર એપ બહાર પાડી."""

In [ ]:
x = tokenizer.tokenize(gujarati_text)

In [ ]:
x

In [ ]:
EOS_TOKEN = tokenizer.eos_token

In [ ]:
from datasets import load_dataset

dataset = load_dataset("rachittshah/alpaca-gujarati", split="train[:3%]")
dataset

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 1559
})

In [ ]:
dataset[0]

{'input': '',
 'instruction': 'સ્વસ્થ રહેવા માટે ત્રણ ટિપ્સ આપો.',
 'output': '૧. સંતુલિત આહાર લો અને ખાતરી કરો કે તેમાં પુષ્કળ ફળો અને શાકભાજી શામેલ છે. \n2. તમારા શરીરને સક્રિય અને મજબૂત રાખવા માટે નિયમિત કસરત કરો. \n૩. પૂરતી ઊંઘ લો અને ઊંઘનું સમયપત્રક સતત જાળવો.'}

In [ ]:
len(dataset)

1559

In [ ]:
output_text = []
for i in range(len(dataset)):
  if dataset[i]['input'] == "":
        text = f"###<start_of_turn>user instruction: {dataset['instruction'][i]}<end_of_turn>\n<start_of_turn>model ### output: {dataset['output'][i]}" + EOS_TOKEN
  else:
        text = f"###<start_of_turn>user instruction: {dataset['instruction'][i]}<end_of_turn>\n<start_of_turn>model ### input: {dataset['input'][i]}\n ### output: {dataset['output'][i]}" + EOS_TOKEN
  output_text.append(text)

In [ ]:
output_text[0]

'###<start_of_turn>user instruction: સ્વસ્થ રહેવા માટે ત્રણ ટિપ્સ આપો.<end_of_turn>\n<start_of_turn>model ### output: ૧. સંતુલિત આહાર લો અને ખાતરી કરો કે તેમાં પુષ્કળ ફળો અને શાકભાજી શામેલ છે. \n2. તમારા શરીરને સક્રિય અને મજબૂત રાખવા માટે નિયમિત કસરત કરો. \n૩. પૂરતી ઊંઘ લો અને ઊંઘનું સમયપત્રક સતત જાળવો.<eos>'

In [ ]:
len(dataset['instruction'])

1559

In [ ]:
len(output_text)

1559

In [ ]:
output_text[0]

'###<start_of_turn>user instruction: સ્વસ્થ રહેવા માટે ત્રણ ટિપ્સ આપો.<end_of_turn>\n<start_of_turn>model ### output: ૧. સંતુલિત આહાર લો અને ખાતરી કરો કે તેમાં પુષ્કળ ફળો અને શાકભાજી શામેલ છે. \n2. તમારા શરીરને સક્રિય અને મજબૂત રાખવા માટે નિયમિત કસરત કરો. \n૩. પૂરતી ઊંઘ લો અને ઊંઘનું સમયપત્રક સતત જાળવો.<eos>'

In [ ]:
dataset = dataset.add_column("prompt", output_text)

In [ ]:
dataset

Dataset({
    features: ['input', 'instruction', 'output', 'prompt'],
    num_rows: 1559
})

In [ ]:
from peft import LoraConfig, PeftModel

In [ ]:
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
lora_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['o_proj', 'q_proj', 'up_proj', 'v_proj', 'k_proj', 'down_proj', 'gate_proj']
)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
trainable, total = model.get_nb_trainable_parameters()
print(f"Trainable: {trainable} | total: {total} | Percentage: {trainable/total*100:.4f}%")

Trainable: 4902912 | total: 2511075328 | Percentage: 0.1953%


In [ ]:
#new code using SFTTrainer
import transformers

from trl import SFTTrainer

tokenizer.pad_token = tokenizer.eos_token
torch.cuda.empty_cache()

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    dataset_text_field="prompt",
    peft_config=lora_config,
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        warmup_steps=0.03,
        max_steps=10,
        learning_rate=2e-4,
        logging_steps=1,
        output_dir="outputs",
        optim="paged_adamw_8bit",
        save_strategy="epoch",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [ ]:
trainer.train()

Step,Training Loss
1,5.497900
2,4.416200
3,5.257500
4,3.821800
5,4.159800
6,4.841200
7,3.070100
8,4.070000
9,3.617000
10,3.209400


TrainOutput(global_step=10, training_loss=4.19609501361847, metrics={'train_runtime': 49.6295, 'train_samples_per_second': 0.806, 'train_steps_per_second': 0.201, 'total_flos': 95711492739072.0, 'train_loss': 4.19609501361847, 'epoch': 0.03})

In [ ]:
new_model = "APP04/gujju-gemma-2b-it-test"

In [ ]:
trainer.model.save_pretrained(new_model)

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map={"": 0},
)
merged_model= PeftModel.from_pretrained(base_model, new_model)
merged_model= merged_model.merge_and_unload()

# Save the merged model
merged_model.save_pretrained("merged_model",safe_serialization=True)
tokenizer.save_pretrained("merged_model")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
print(merged_model)

GemmaForCausalLM(
  (model): GemmaModel(
    (embed_tokens): Embedding(256000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-17): 18 x GemmaDecoderLayer(
        (self_attn): GemmaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): GemmaRotaryEmbedding()
        )
        (mlp): GemmaMLP(
          (gate_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (up_proj): Linear(in_features=2048, out_features=16384, bias=False)
          (down_proj): Linear(in_features=16384, out_features=2048, bias=False)
          (act_fn): GELUActivation()
        )
        (input_layernorm): GemmaRMSNorm()
        (post_attention_layernorm): GemmaRMSNorm()
      )
    )
    (norm): GemmaRMSNor

In [ ]:
from huggingface_hub import login
login()

In [ ]:
merged_model.push_to_hub('APP04/gujju-gemma-2b-it-test', use_temp_dir=False)
tokenizer.push_to_hub('APP04/gujju-gemma-2b-it-test', use_temp_dir=False)

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/APP04/APP04/commit/255b8889929c461b185084b153cdef83006edb6d', commit_message='Upload tokenizer', commit_description='', oid='255b8889929c461b185084b153cdef83006edb6d', pr_url=None, pr_revision=None, pr_num=None)